In [1]:
import numpy as np
import pandas as pd
import os
import math

In [2]:
def read_data(src):
    globals()[f'dvars_{src}'] = []
    for i in os.listdir(f"data/{src}/"):
        path = i
        i = i.replace('DSS_9AB', 'DAB').replace('9AB', 'AB')
        j = i[:i.index(f'({src[0].upper()})')].split('+')
        day = ''
        treatm = ''
        if len(j) == 1:
            day = j[0][:j[0].index('D')]
            treatm = j[0].split(' ')[1].lower()
        else:
            day = '15'
            treatm = j[0].split(' ')[1].lower() + '_' + j[1].split(' ')[1].lower()
        dvar = f'{src}_{day}_{treatm}'.replace('h20', 'h2o')
        globals()[f'dvars_{src}'].append([dvar, day, treatm])
        globals()[dvar] = pd.read_excel(f"data/{src}/{path}",
                                        sheet_name=None)

In [3]:
def split_data():
    globals()["dvars_cecal_mouse_1"] = []
    globals()["dvars_cecal_mouse_2"] = []
    globals()["dvars_serum_mouse_1"] = []
    globals()["dvars_serum_mouse_2"] = []
    for row in dvars_cecal:
        i = row[0]
        for j in [1, 2]:
            globals()[f"{i}_mouse_{j}"] = globals()[i][list(globals()[i].keys())[j-1]]
            globals()[f"dvars_cecal_mouse_{j}"].append([f"{i}_mouse_{j}", row[1], row[2]])
    for row in dvars_serum:
        i = row[0]
        for j in [1, 2]:
            globals()[f"{i}_mouse_{j}"] = globals()[i][list(globals()[i].keys())[j-1]]
            globals()[f"dvars_serum_mouse_{j}"].append([f"{i}_mouse_{j}", row[1], row[2]])

In [4]:
read_data("serum")
read_data("cecal")
split_data()

In [5]:
def save_data_reps(davars, name):
    for r in [0, 1, 2]:
        df = pd.DataFrame(columns=['Metabolite Name'] + [x[0] for x in davars])
        cmdfs = [globals()[i]['Metabolite Name'] for i in [x[0] for x in davars]]
        mdf = pd.concat(cmdfs).drop_duplicates().reset_index(drop=True)
        df['Metabolite Name'] = mdf
        for i in [x[0] for x in davars]:
            cdf = globals()[i][['Metabolite Name', f'rep_{r}']]
            df = df.merge(cdf, on='Metabolite Name', how='left')
            df[i] = df[f'rep_{r}'].fillna(df[i]).astype(float)
            df = df.drop(f'rep_{r}', axis=1)
        df = df.fillna(0)
        seriesmet = df['Metabolite Name']
        df = df.drop('Metabolite Name', axis = 1)
        df = df.rename(columns = lambda x: x[6:-8])
        df['Metabolite Name'] = seriesmet
        df.to_hdf(f"generated/a_{name}.hdf5", f'rep_{r}')

In [6]:
save_data_reps(dvars_cecal_mouse_1, 'cecal_mouse_1')
save_data_reps(dvars_serum_mouse_1, 'serum_mouse_1')
save_data_reps(dvars_cecal_mouse_2, 'cecal_mouse_2')
save_data_reps(dvars_serum_mouse_2, 'serum_mouse_2')